# **Creating your First Plug-In**
This notebook guides you through the process of creating your first Accelergy
plug-in.

In [1]:
from creating_first_plug_in.helper_functions import *
import os
CURRENT_DIR = os.path.dirname(os.path.realpath('__file__'))
PLUG_IN_SCRIPT = 'my_plugin.py'
LOGFILE = 'accelergy.log'

## **Introducing the Ternary MAC Plug-In**
We'd like to build a custom MAC unit for ternary neural nets. Ternary neural
    nets restrict input activations and weight values to [-1, 0, 1]. This
    allows them to compute MACs with extremely simple hardware. Each
    $Input\times Weight$ value is also in [-1, 0, 1], so a MAC can be computed
    with a single increment/decrement of the psum. Suppose that, based on
    simulations, we have the following formulas for the energy and area of the
    ternary MAC unit: $$MAC\ Energy (pJ) = 0.002\times(Accum.\ Datawidth +
    0.25) \times (Tech\ Node)^{1.1} $$ $$Area\ (um^2) = 2\times(Accum.\
    Datawidth + 0.3) \times (Tech\ Node)^{1.3} $$

We also set a few restrictions on our model: $$ 4\le Accum.\ Datawidth \le 8 $$
    $$ 16\le Tech\ Node \le 130 $$


Let's build an Accelergy plug-in to implement our new MAC unit. The plug-in
below is commented to explain each part. All "REQUIRED" pieces must be included
for the plug-in to work.

In [2]:
from accelergy.plug_in_interface.estimator import (
    Estimator, actionDynamicEnergy, add_estimator_path, remove_estimator_path
)

# REQUIRED: Declare a new Accelergy Estimator
class TernaryMAC(Estimator):
    # REQUIRED: Tell Accelergy the name of this Estimator
    name = 'ternary_mac'
    # REQUIRED: Tell Accelergy the accuracy of this Estimator
    percent_accuracy_0_to_100 = 80

    def __init__(self, accum_datawidth: int, tech_node: int):
        self.accum_datawidth = accum_datawidth
        self.tech_node = tech_node
        # Raising an error tells Accelergy that this plug-in can't estimate,
        # and Accelergy should query other plug-ins instead. Good error
        # messages are essential for users debugging Accelergy designs.
        assert 4 <= accum_datawidth <= 8, \
            f'Accumulation datawidth {accum_datawidth} outside supported ' \
            f'range [4-8]!'
        assert 16 <= tech_node <= 130, \
            f'Technology node {tech_node} outside supported range [16, 130]!'

    # The actionDynamicEnergy decorator tells Accelergy that this function should be
    # made visible as an action. The function should return an energy in
    # Joules.
    @actionDynamicEnergy
    def mac_random(self) -> float:
        self.logger.info(f'TernaryMAC Estimator is estimating '
                         f'energy for mac_random.')
        return 0.002e-12 * (self.accum_datawidth + 0.25) * self.tech_node**1.1

    # REQUIRED: The get_area function returns the area of this component. It is
    # required in all plug-ins. The function should return an area in m$^2$.
    def get_area(self) -> float:
        self.logger.info(f'TernaryMAC Estimator is estimating area.')
        return 2 * (self.accum_datawidth + 0.3) * self.tech_node ** 1.3 * 1e-12
    
    # REQUIRED: The get_latency function returns the latency of this component.
    def leak(self, global_cycle_seconds: float) -> float:
        """ Returns the leakage energy per global cycle or an Estimation object 
        with the leakage energy and units. """
        return 1e-3 * global_cycle_seconds # 1mW

# Ignore the next line. Moves the plug-in to a script because we're in a
# Jupyter notebook.
plugin_notebook2script(TernaryMAC, PLUG_IN_SCRIPT) 
add_estimator_path(PLUG_IN_SCRIPT) # Install plug-in so Accelergy can find it.

2024-02-03 10:09:20 INFO        Located config file at /home/tanner/.config/accelergy/accelergy_config.yaml.
2024-02-03 10:09:20 INFO        Located config file at /home/tanner/.config/accelergy/accelergy_config.yaml.
2024-02-03 10:09:20 INFO        Added path /home/tanner/research/cimloop/infrastructure/timeloop-accelergy-exercises/workspace/exercises/04_creating_accelergy_plug_ins/my_plugin.py to the list of python paths.


Writing TernaryMAC. View the script at /home/tanner/research/cimloop/infrastructure/timeloop-accelergy-exercises/workspace/exercises/04_creating_accelergy_plug_ins/my_plugin.py


## **Plug-In Essential Pieces**

Accelergy plug-ins are identified by inheriting the Estimator class from
accelergy.plug_in_interface.estimator. All Estimator classes defined in a
module are automatically imported by Accelergy and made available to Accelergy.

**The following are the essential pieces for an Accelergy Estimator class:**
- The class must inherit from Estimator
- The class must have a `name` attribute. `name` can either be a string or a
  list of strings. If the `name` attribute matches the `class` of an Accelergy
  design component, the plug-in may be used to estimate the area/energy of that
  component.
- The class must have a `percent_accuracy_0_to_100` attribute. This is a number
  between 0 and 100 that tells Accelergy how accurate the plug-in is. It
  determines a plug-in's priority when multiple plug-ins are available for a
  component.
- The class must have a get_area function. This function should return the area
  of the component in m$^2$. This function should not have any arguments.
- Any number of functions may be decorated with `@actionDynamicEnergy`. These
  functions are named the name of an action, and they return the energy of that
  action in Joules. These functions may have any number of arguments.

Modules with plug-ins can be installed with the add_estimator_path function,
which adds a path for Accelergy to search. The remove_estimator_path can remove
paths.

### Let's see our plug-in estimate the area and energy of the ternary MAC unit

In [3]:
TARGET_YAML = os.path.join(CURRENT_DIR, 'creating_first_plug_in/design0.yaml')
# Check the YAML file we'll be using to query our new plug-in
print('YAML file to query our new plug-in:')
print(yaml_section(TARGET_YAML, ['compound_components']))

# Run Accelergy to generate the output directory
print(f'\nRunning Accelergy...')
!accelergy {TARGET_YAML} -o out_dir > accelergy.log 2>&1

# Check the output of Accelergy
READ_LOGGED = 'ternary_mac(tech_node=45, accum_datawidth=5)'
print(f'\nAccelergy log from TernaryMAC plug-in:')
print(get_log_lines(LOGFILE, READ_LOGGED))

YAML file to query our new plug-in:
	version: 0.4
	classes:
	  - name: custom_mac
	    attributes: {}
	    subcomponents:
	      - name: mac
	        class: ternary_mac
	        attributes:
	            tech_node: 45
	            accum_datawidth: 5
	    actions:
	      - name: mac_random
	        subcomponents:
	          - name: mac
	            actions:
	              - name: mac_random

Running Accelergy...

Accelergy log from TernaryMAC plug-in:



## **Area/Energy Estimations**
Accelergy will output an Area Reference Table (ART) and Energy Reference Table
(ERT) with area/energy breakdowns of the architecture. The estimations from our
plug-in, alongside those of other plug-ins, will be used to fill this table.

#### **Estimating Area of a Component**
The `get_area` function is called on the Estimator instance to get the area of
the component. This function should not take any arguments. If the Estimator
class can not estimate the area, it should raise an error. This will cause
Accelergy to look for another Estimator. Accelergy will report the error if no
other Estimator is found.

#### **Estimating Energy of an Action**
If an Estimator has been instantiated for a component, Accelergy will search for
functions decorated with `@actionDynamicEnergy`. If the name of the function
matches the name of an action, the function will be used to estimate energy.

If the Accelergy design specifies arguments for an action, the arguments will
be passed to the function. Extra arguments are ignored, but missing arguments
will cause a failed match. If an argument has a default value, the default
value will be used if the argument is not specified in the Accelergy design.

If the function can not estimate the area, it should raise an error. This will
cause Accelergy to look for another Estimator. Accelergy will report the error
if no other Estimator is found.


In [4]:
# Find the estimations in the output ERT
print(f'\nAccelergy ERT with TernaryMAC estimations. Reported numbers in pJ:')
print(yaml_section('./out_dir/ERT.yaml', ['ERT', 'tables', -1]))
print(f'\nAccelergy ART with TernaryMAC estimations. Reported numbers in um$^2$:')
print(yaml_section('./out_dir/ART.yaml', ['ART', 'tables', -1]))


Accelergy ERT with TernaryMAC estimations. Reported numbers in pJ:


ValueError: Error loading YAML file /home/tanner/research/cimloop/infrastructure/timeloop-accelergy-exercises/workspace/exercises/04_creating_accelergy_plug_ins/out_dir/ERT.yaml

## **Instantiating Plug-Ins**
When Accelergy parses architectures, it will generate a list of components and
attributes. Additionally, each component will have associated actions.
Accelergy searches for the plug-in with the highest accuracy to estimate the
area of each component and the energy of each action.

An instance the Estimator class is created for a component if the following
conditions are met:
- The `class` attribute of the component matches the `name` attribute of the
  Estimator class
- The attributes of the component match the arguments of the Estimator class's
  \_\_init\_\_ function. Extra attributes are ignored, but missing attributes
  will cause a failed match.
- This is the highest-accuracy matching Estimator available to Accelergy.

Let's try putting together a few plug-ins and see which ones Accelergy picks.
We'll set up two plug-ins:
- The AccurateButPicky Estimator has a very high accuracy, but only supports
  signed operands for its MAC operation. It also does not have a reset
  operation.
- The InaccurateFlexible Estimator has a lower accuracy, but supports both
  signed and unsigned operands. It also has a reset operation with an optional
  argument make_expensive that makes the reset operation *very*
  energy-intensive.

In [ ]:
class AccurateButPicky(Estimator):
    name = 'ternary_mac'
    percent_accuracy_0_to_100 = 99 # This plug-in is VERY confident

    def __init__(self, accum_datawidth: int, tech_node: int):
        self.accum_datawidth = accum_datawidth
        self.tech_node = tech_node

    @actionDynamicEnergy
    def mac_random(self, unsigned: bool) -> float:
        assert not unsigned, 'Sorry, I only support signed operands.'
        return 0.002e-12 * (self.accum_datawidth + 0.25) * self.tech_node**1.1

    def get_area(self) -> float:
        return 2e-12 * (self.accum_datawidth + 0.3) * self.tech_node**1.3

    def leak(self, global_cycle_seconds: float) -> float:
        return 1e-3 * global_cycle_seconds # 1mW

class InacurrateFlexible(Estimator):
    name = 'ternary_mac'
    percent_accuracy_0_to_100 = 50 # This plug-in is not very confident

    def __init__(self, accum_datawidth: int, tech_node: int):
        self.accum_datawidth = accum_datawidth
        self.tech_node = tech_node

    @actionDynamicEnergy
    def mac_random(self, unsigned: bool) -> float:
        energy = 0.002e-12 * (self.accum_datawidth + 0.25) * \
                 self.tech_node**1.1
        if unsigned:
            self.logger.info('Unsigned mac_random consuming half the energy.')
            energy /= 2
        return energy
    
    @actionDynamicEnergy
    def reset(self, make_expensive: bool=False) -> float:
        if make_expensive:
            self.logger.info('InacurrateFlexible reset is expensive! '
                             'Returning a high energy.')
            return 1 # 1 Joule
        return 2e-12 * self.tech_node ** 1.1
    
    def get_area(self) -> float:
        return 2 * (self.accum_datawidth + 0.3) * self.tech_node ** 1.3 * 1e-12

    def leak(self, global_cycle_seconds: float) -> float:
        return 1e-3 * global_cycle_seconds # 1mW

# Ignore this line. Moves the plug-in to the script.
plugin_notebook2script([AccurateButPicky, InacurrateFlexible], PLUG_IN_SCRIPT)


## **Plug-In Precedence and Function Calling Example**
A new compound component is set up to query our two plug-ins. It has four
actions:
- mac_random_to_accurate_picky: This component+action is set up to fit the
  AccurateButPicky Estimator.
- mac_random_to_inaccurate_flexible: Accelergy first asks the AccurateButPicky
  estimator, but the picky estimator raises an error. Accelergy queries the
  InaccurateFlexible estimator.
- reset_inexpensive_default_argument: Only the InaccurateFlexible estimator has
  a reset action. The estimator has a make_expensive argument with a default of
  0. The make_expensive argument is not specified in the YAML so is set to its
  default of 0.
- reset_expensive_specified_argument: The make_expensive argument is specified
  in the YAML and overrides the default value from the InaccurateFlexible
  Estimator. Reset is very expensive indeed.

In [ ]:
TARGET_YAML = os.path.join(CURRENT_DIR, 'creating_first_plug_in/design1.yaml')

# Check the YAML file we'll be using to query our new plug-in
print('YAML file to query our new plug-in:')
print(yaml_section(TARGET_YAML, ['compound_components']))

# Run Accelergy to generate the output directory
print(f'\nRunning Accelergy...')
!accelergy {TARGET_YAML} -v -o out_dir > accelergy.log 2>&1

# Find the estimations in the output ERT
print(f'\nAccelergy ERT with TernaryMAC estimations. Reported numbers in pJ:')
print(yaml_section('./out_dir/ERT_summary_verbose.yaml', 
                   ['ERT_summary', 'table_summary', -1, 'primitive_estimation(s)'], 
                   ['name', 'action_name', 'arguments', 'energy', 'estimator']))

## **Interpreting Estimator Logs**
Estimator classes communicate with the user in two ways:
- If an estimator class can not be instantiated or can not estimate something,
  it should raise an error. The error message is shown in the Accelergy log
  file if no other Estimator can be found.
- Each Estimator class has a logger attribute. This is a Python logger that
  includes info(message), warning(message), and error(message) functions. If an
  estimator is used to estimate something, the logger's messages are shown in
  the Accelergy log file depending on the verbosity level.

By peeking at Accelergy's output, we can see the procedure by which Accelergy
finds estimators. Let's see what Accelergy tells us about the
ternary_mac.mac_random(unsigned=1) action:
1. Accelergy found that both the InacurrateFlexible and AccurateButPicky
   estimators have a matching name and action mac_random(unsigned).
2. Accelergy asks the higher-accuracy plug-in AccurateButPicky to estimate the
   energy.
3. The AccurateButPicky plug-in raises an error because the unsigned argument
   is not False.
4. Accelergy asks the lower-accuracy plug-in InacurrateFlexible to estimate the
   energy.
5. The InacurrateFlexible plug-in estimates the energy and returns it to
   Accelergy.

Generally, Accelergy will report logs from the plug-in used to estimate
energy/area. It will also report errors if a plug-in tried to estimate
energy/area but failed. To see the logs from all installed plug-ins, we can run
Accelergy with the -v flag. This will cause Accelergy to print the reason why
each and every plug-in was or was not used to estimate energy/area.

In [ ]:
READ_LOGGED = ['ENERGY ESTIMATION', 'ternary_mac', 'mac_random']

print(f'Running Accelergy with standard logging...')
!accelergy {TARGET_YAML} -o out_dir > accelergy.log 2>&1
print(get_log_lines(LOGFILE, READ_LOGGED))

print(f'\n\nRunning Accelergy with verbose logging...')
!accelergy {TARGET_YAML} -v -o out_dir > accelergy.log 2>&1
print(get_log_lines(LOGFILE, READ_LOGGED))

## **Cleaning Up**
We're all done! Let's uninstall the plug-in and create a template for future plug-ins.

In [ ]:
remove_estimator_path(PLUG_IN_SCRIPT)

## **Template for Future Plug-Ins**


In [ ]:
import os
from accelergy.plug_in_interface.estimator import (
    Estimator, actionDynamicEnergy, add_estimator_path, remove_estimator_path
)

class PlugInTemplate(Estimator):
    name = 'component_name or list of names'
    percent_accuracy_0_to_100 = 50

    def __init__(self, arg0: int, arg1: int=5):
        self.logger.info('The __init__ function is called if the name and '
                         'required arguments match.')
        assert arg0 > 0, 'Raise an error if the arguments are not valid.'
        self.arg0, self.arg1 = arg0, arg1

    @actionDynamicEnergy
    def action(self, arg0: int, arg1: int=5) -> float:
        self.logger.info('@actionDynamicEnergy can decorate any number of actions.')
        assert arg1 > 0, 'Raise an error if we can not estimate the energy.'
        return arg0 * arg1 * 1e-12 # Return energy in Joules

    def get_area(self) -> float:
        self.logger.info('The get_area function is required.')
        assert self.arg1 > 0, 'Raise an error if we can not estimate the area.'
        return self.arg0 * self.arg1 * 1e-12 # Return area in m$^2$

    @staticmethod
    def install_this_file():
        add_estimator_path(os.path.abspath(__file__))

    @staticmethod
    def uninstall_this_file():
        remove_estimator_path(os.path.abspath(__file__))


print(f'Thank you for completing the tutorial!')